# Task - 3 Exploratory Data Analysis - Terrorism

Importing dependencies

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import io
import base64
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')
import codecs
import plotly.express as px 

Importing Dataset

In [ ]:
dataset = pd.read_csv("globalterrorismdb_0718dist.csv", encoding='latin1')

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset.shape

In [ ]:
dataset.columns.values

Keeping the necessary features and excluding the others

In [ ]:
dataset.rename(columns={'iyear':'Year','imonth':'Month','city':'City','iday':'Day','country_txt':'Country','region_txt':'Region','attacktype1_txt':'AttackType','target1':'Target','nkill':'Killed','nwound':'Wounded','summary':'Summary','gname':'Group','targtype1_txt':'Target_type','weaptype1_txt':'Weapon_type','motive':'Motive', 'success': 'Success'},inplace=True)
dataset['Casualities'] = dataset.Killed + dataset.Wounded
dataset=  dataset[['Year','Month','Day','Country','Region','City', 'Success','latitude','longitude','AttackType','Killed','Wounded','Casualities','Target','Group','Target_type','Weapon_type']]


In [ ]:
dataset.head()

Checking for the null values 

In [ ]:
dataset.isnull().sum()

Dealing with the nan values

In [ ]:
dataset["City"].fillna("Unknown", inplace = True)
dataset["Killed"].fillna(0, inplace = True)
dataset["Target"].fillna("Unknown", inplace = True)

In [ ]:
dataset.isnull().sum()

# Exploratory Data Analysis (EDA)

# ANNUAL PLOT

Visualizing attacks per year with CountPlot

In [ ]:
plt.subplots(figsize = (15,6))
sns.countplot('Year', data = dataset, palette = 'RdYlGn_r', edgecolor = sns.color_palette('dark',10))
plt.xticks(rotation = 90)
plt.title('Number Of Terrorist Activities Each Year')
plt.show()

# ANNUAL CALSUALTIES

In [ ]:
plt.subplots(figsize = (15,6))
year_cas = dataset.groupby('Year').Casualities.sum().to_frame().reset_index()
year_cas.columns = ['Year','Casualities']
sns.barplot(x = year_cas.Year, y = year_cas.Casualities, palette = 'RdYlGn_r', edgecolor = sns.color_palette('dark',10))
plt.xticks(rotation = 90)
plt.title('Number Of Casualities Each Year')
plt.show()

# Success vs Failure

Visualizing the attacks per year with success and failures.

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(x = "Year", hue = "Success", data = dataset)
plt.title("Success vs Failure")

# Number of Attacks in Each Country

In [ ]:
plt.subplots(figsize=(15,6))
country_attacks = dataset.Country.value_counts()[:15].reset_index()
country_attacks.columns = ['Country', 'Total Attacks']
sns.barplot(x = country_attacks.Country, y = country_attacks['Total Attacks'], palette= 'OrRd_r',edgecolor = sns.color_palette('dark',10))
plt.xticks(rotation=30)
plt.title('Number of Attacks in Each Country')
plt.show()

# Number of Casualties in Each Country

In [ ]:
plt.subplots(figsize = (15,6))
count_cas = dataset.groupby('Country').Casualities.sum().to_frame().reset_index().sort_values('Casualities', ascending=False)[:15]
sns.barplot(x = count_cas.Country, y = count_cas.Casualities, palette =  'OrRd_r',edgecolor = sns.color_palette('dark',10))
plt.xticks(rotation=30)
plt.title('Number of Casualities in Each Country')
plt.show()

# Number of Attacks in each Region

In [ ]:
region_attacks = dataset.Region.value_counts().to_frame().reset_index()
region_attacks.columns = ['Region', 'Total Attacks']
plt.subplots(figsize = (15,6))
sns.barplot(x = region_attacks.Region, y = region_attacks['Total Attacks'], palette = 'OrRd_r', edgecolor = sns.color_palette('dark',10))
plt.xticks(rotation = 90)
plt.title('Number of Attacks in Each Region')
plt.show()

# Number of Casualties in Each Region

In [ ]:
region_casualties = dataset.Region.value_counts().to_frame().reset_index()
region_casualties.columns = ['Region', 'Total Casualties']
plt.subplots(figsize = (15,6))
sns.barplot(x = region_casualties.Region, y = region_casualties['Total Casualties'], palette = 'OrRd_r', edgecolor = sns.color_palette('dark',10))
plt.xticks(rotation = 90)
plt.title('Number of Casualties in Each Region')
plt.show()

In [ ]:
figure = px.scatter(dataset, x="Year", y="Region", color="AttackType",size="Killed", hover_name="Country", log_x=True, size_max=80, title = "Attack in Countries in each Year")

figure.show()

In [ ]:
px.pie(dataset, names = "AttackType", values = "Killed", title = "Percentage of Kills with specific Weapon")

In [ ]:
px.pie(dataset, names = "Target_type", values = "Killed", title = "Percentage of Kills on the Target")

# World Map

In [ ]:
world = dataset.head(len(dataset))
fig = px.scatter_mapbox(world, lat='latitude', lon='longitude', hover_name="City", hover_data=["Weapon_type",'Country','Success'],
                        color_discrete_sequence=["fuchsia"], zoom=1, height=500)
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()